![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# Classificação de Atividade Humana com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [130]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics            import confusion_matrix
from sklearn.metrics            import ConfusionMatrixDisplay


In [131]:


filename_features = "base/features.txt"
filename_labels = "base/activity_labels.txt"

filename_subtrain = "base/train/subject_train.txt"
filename_xtrain = "base/train/X_train.txt"
filename_ytrain = "base/train/y_train.txt"

filename_subtest = "base/test/subject_test.txt"
ffilename_xtest = "base/test/X_test.txt"
filename_ytest = "base/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze()
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze()
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze()
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## PCA com variáveis padronizadas

Reflexão sobre a escala das variáveis:

**Variáveis em métricas muito diferentes** podem interferir na análise de componentes principais. Lembra que variância é informação pra nós? Pois bem, tipicamente se há uma variável monetária como salário, vai ter uma ordem de variabilidade bem maior que número de filhos, tempo de emprego ou qualquer variável dummy. Assim, as variáveis de maior variância tendem a "dominar" a análise. Nesses casos é comum usar a padronização das variáveis.

Faça duas análises de componentes principais para a base do HAR - com e sem padronização e compare:

- A variância explicada por componente
- A variância explicada acumulada por componente
- A variância percentual por componente
- A variância percentual acumulada por componente
- Quantas componentes você escolheria, em cada caso para explicar 90% da variância?

In [132]:
def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean())/s.std()
    return s

X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)
X_test_pad = pd.DataFrame(X_test).apply(padroniza, axis=0)

In [142]:
def analise_pca(data:pd.DataFrame, porcentagem:float=0.95,text:bool=False) -> tuple[pd.DataFrame, int]:
    '''
    Função que realiza a análise de componentes principais de um DataFrame

    data: DataFrame - DataFrame a ser analisado

    porcentagem: float (default=0.95) - Porcentagem da variância que se deseja explicar

    return: DataFrame com os componentes principais e a quantidade de componentes necessários para explicar a variância desejada    
    '''
    pca = PCA()
    principalComponents = pca.fit(data)
    variancia_explicada             = principalComponents.explained_variance_[:5]
    variancia_explicada_acumulada   = principalComponents.explained_variance_.cumsum()[:5]
    variancia_percentual            = principalComponents.explained_variance_ratio_[:5]
    variancia_percentual_acumulada  = principalComponents.explained_variance_ratio_.cumsum()[:5]
    quantidade_componentes = (principalComponents.explained_variance_ratio_.cumsum()<porcentagem).sum()+1

    if text:
        print('## A seguir amostras da variância dos componentes principais ##\n')
        print(f"Variância explicada:\n {variancia_explicada}\n")
        print(f"Variância explicada acumulada:\n {variancia_explicada_acumulada}\n")
        print(f"Variância percentual:\n {variancia_percentual}\n")
        print(f"Variância percentual acumulada:\n {variancia_percentual_acumulada}\n")

        print(f"Quantidade de componentes para explicar {porcentagem*100}% da variância: {quantidade_componentes}")

    compontentes = principalComponents.transform(data)
    compontentes_df = pd.DataFrame(compontentes,columns=data.columns)
    
    return compontentes_df,quantidade_componentes


In [143]:
## Aplicando a função de análise de PCA para base padronizada
df_train_pad, n_componente_pad = analise_pca(X_train_pad,0.90,True)

## A seguir amostras da variância dos componentes principais ##

Variância explicada:
 [284.88237655  36.9176163   15.74411031  14.0471749   10.59327893]

Variância explicada acumulada:
 [284.88237655 321.79999285 337.54410316 351.59127806 362.18455699]

Variância percentual:
 [0.50781172 0.0658068  0.02806437 0.02503953 0.01888285]

Variância percentual acumulada:
 [0.50781172 0.57361853 0.60168289 0.62672242 0.64560527]

Quantidade de componentes para explicar 90.0% da variância: 63


In [144]:
## Aplicando a função de análise de PCA para base não padronizada
df_train, n_componente = analise_pca(X_train,0.90,True)

## A seguir amostras da variância dos componentes principais ##

Variância explicada:
 [34.82363041  2.73504627  2.29439284  1.04377529  0.943517  ]

Variância explicada acumulada:
 [34.82363041 37.55867667 39.85306951 40.89684481 41.84036181]

Variância percentual:
 [0.6255444  0.04913023 0.04121467 0.01874956 0.0169486 ]

Variância percentual acumulada:
 [0.6255444  0.67467463 0.7158893  0.73463886 0.75158746]

Quantidade de componentes para explicar 90.0% da variância: 34


## Árvore com PCA

Faça duas uma árvore de decisão com 10 componentes principais - uma com base em dados padronizados e outra sem padronizar. Utilize o ```ccp_alpha=0.001```.

Compare a acurácia na base de treino e teste.

In [145]:
def treino_componemtes(
        _X_train:pd.DataFrame,
        _X_test:pd.DataFrame,
        _y_train:pd.DataFrame,
        _y_test:pd.DataFrame,
        n:int=0):
    '''
    Função que realiza o treinamento de um modelo de árvore de decisão com base em um número de componentes principais
    '''
    pca = PCA(n_components=n)
    pc_treino = pca.fit_transform(_X_train)
    pc_teste = pca.transform(_X_test)
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(pc_treino, _y_train)
    print(f'acurácia de treino = {clf.score(pc_treino, _y_train)*100:.2f}%')
    print(f'acurácia de teste = {clf.score(pc_teste, _y_test)*100:.2f}%')

In [146]:
## Treinando o modelo com a base padronizada
treino_componemtes(X_train_pad,X_test_pad,y_train,y_test,10)


acurácia de treino = 85.92%
acurácia de teste = 77.74%


In [147]:
## Treinando o modelo com a base não padronizada
treino_componemtes(X_train,X_test,y_train,y_test,10)


acurácia de treino = 89.35%
acurácia de teste = 82.32%


**Conclusão**
Os testes com a base `não` padronizada apresentam melhores indicadores de acurácia e quantidade menor de componentes necessários para explicação